### Masked image augmentation demo

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0,
        height_shift_range=0,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data_simple_cnn/train/mask/1_mask.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (70, 116, 3)

print(x.shape)

x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='aug_preview', save_prefix='mask', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

### Simple CNN model architecture for classification

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

simple_cnn = Sequential()
simple_cnn.add(Conv2D(16, (3, 3), input_shape=(70, 116, 1), padding='same', activation='relu', name='conv1_1'))
simple_cnn.add(Conv2D(16, (3, 3), input_shape=(70, 116, 1), padding='same', activation='relu', name='conv1_2'))
simple_cnn.add(MaxPooling2D(pool_size=(2, 2), name='pool1'))

simple_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu', name='conv2_1'))
simple_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu', name='conv2_2'))
simple_cnn.add(MaxPooling2D(pool_size=(5, 2), name='pool2'))

simple_cnn.add(Flatten(name='flatten'))
simple_cnn.add(Dense(32, activation='relu', name='fc1'))
simple_cnn.add(Dropout(0.5))
simple_cnn.add(Dense(1, activation='sigmoid', name='prediction'))

simple_cnn.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


In [ ]:
simple_cnn.summary()

### Read small image files for classification

In [2]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_datagen = ImageDataGenerator(
        rotation_range=180,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rotation_range=180, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        'data_simple_cnn/train',  
        target_size=(70, 116), 
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale') 

validation_generator = test_datagen.flow_from_directory(
        'data_simple_cnn/validation',
        target_size=(70, 116),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale')

Found 14400 images belonging to 2 classes.
Found 3570 images belonging to 2 classes.


### Classification training for small images

In [ ]:
simple_cnn.fit_generator(
        train_generator,
        steps_per_epoch=224 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=112/ batch_size)
simple_cnn.save_weights('simple_cnn.h5')

### Image segmention loss function

In [ ]:
import keras.backend as K

def binary_crossentropy_with_logits(ground_truth, predictions):
    return K.mean(K.binary_crossentropy(ground_truth,
                                        predictions,
                                        from_logits=True),
                  axis=-1)

### FCN model architecture for small image segmentation

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Add, Dropout

'''
simple_cnn = Sequential()
simple_cnn.add(Conv2D(32, (3, 3), input_shape=(70, 116, 1), padding='same', activation='relu', name='conv1'))
simple_cnn.add(MaxPooling2D(pool_size=(2, 2), name='pool1'))

simple_cnn.add(Conv2D(64, (3, 3), padding='same', activation='relu', name='conv2'))
simple_cnn.add(MaxPooling2D(pool_size=(5, 2), name='pool2'))

simple_cnn.add(Flatten(name='flatten'))
simple_cnn.add(Dense(64, activation='relu', name='fc1'))
simple_cnn.add(Dropout(0.5))
simple_cnn.add(Dense(1, activation='sigmoid', name='prediction'))
'''
def fcn_cnn():
    fc_cnn = Sequential()
    fc_cnn.add(Conv2D(16, (3, 3), input_shape=(70, 116, 1), padding='same', activation='relu', name='conv1_1'))
    fc_cnn.add(Conv2D(16, (3, 3), input_shape=(70, 116, 1), padding='same', activation='relu', name='conv1_2'))
    fc_cnn.add(MaxPooling2D(pool_size=(2, 2), name='pool1'))

    fc_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu', name='conv2_1'))
    fc_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu', name='conv2_2'))
    fc_cnn.add(MaxPooling2D(pool_size=(5, 2), name='pool2'))

    # continue to use convoluational layers instead of fully connected layers
    fc_cnn.add(Conv2D(128, (7, 29), padding='same', activation='relu', name='fc3'))
    fc_cnn.add(Dropout(0.5))
    fc_cnn.add(Conv2D(128, (1, 1), padding='same', activation='relu', name='fc4'))
    fc_cnn.add(Dropout(0.5))

    # remove original final sigmoid classifier
    fc_cnn.add(Conv2D(2, (1, 1), padding='same', name='logit_fc4')) # [7, 29, 2]
    # deconv logit_fc4 by [5, 2] to [35, 58, 2]
    fc_cnn.add(Conv2DTranspose(2, kernel_size=(2*5-5%2, 2*2-2%2), strides=(5, 2), padding='same', name='deconv_logit_fc4'))
    # conv logit from pool1 to [35, 58, 2]

    # add deconv_logits_fc4 and logit_pool1
    logit_pool1 = Conv2D(2, (1, 1), padding='same', name='logit_pool1')(fc_cnn.layers[2].output)
    logit_pool1_deconv_logit_fc4 = Add()([logit_pool1, fc_cnn.layers[-1].output])
    # deconv above sum by [2, 2] to [70, 116, 2] --> pixel-wise classification: segmentation logit
    final_deconv = Conv2DTranspose(2, kernel_size=(2*2-2%2, 2*2-2%2), strides=(2, 2), 
                                   padding='same', name='final_deconv')(logit_pool1_deconv_logit_fc4)
    
    return Model(fc_cnn.input, final_deconv)

fcn = fcn_cnn()

# need to change loss for image and mask logit cross entropy
fcn.compile(loss=binary_crossentropy_with_logits,
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
simple_cnn.summary()

In [ ]:
layers = simple_cnn.layers
for i in range(len(layers)):
    n = len(layers[i].get_weights())
    print(str(i) + "-layer weight len: ", n)
    if n == 2:
        print("  weight matrix size: ", layers[i].get_weights()[0].shape)
        print("  bias vector size: ", layers[i].get_weights()[1].shape)
        print(layers[i].get_weights()[0])

In [ ]:
fcn.summary()

In [ ]:
layers = fcn.layers
for i in range(len(layers)):
    n = len(layers[i].get_weights())
    print(str(i) + "-layer weight len: ", n)
    if n == 2:
        print("  weight matrix size: ", layers[i].get_weights()[0].shape)
        print("  bias vector size: ", layers[i].get_weights()[1].shape)
        print(layers[i].get_weights()[0])

### Initialize weights in fcn from those in simple_cnn

In [ ]:
def set_weights(fcn, simple_cnn):
    for i in range(5):
        fcn.layers[i+1].set_weights(simple_cnn.layers[i].get_weights())
        
set_weights(fcn, simple_cnn)

In [ ]:
layers = fcn.layers
for i in range(len(layers)):
    n = len(layers[i].get_weights())
    print(str(i) + "-layer weight len: ", n)
    if n == 2:
        print("  weight matrix size: ", layers[i].get_weights()[0].shape)
        print("  bias vector size: ", layers[i].get_weights()[1].shape)
        print(layers[i].get_weights()[0])

### Read small image and mask files for segmentation

In [ ]:
import skimage.io as io
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=180.,
                     horizontal_flip=True,
                     fill_mode='nearest')
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1
imgs_filename = ["data_fcn/train/images/images/"+str(i)+".jpg" for i in range(1, 10)]
masks_filename = ["data_fcn/train/masks/masks/"+str(i)+"_mask.jpg" for i in range(1, 10)]
sample_imgs = [np.expand_dims(io.imread(img_name), -1) for img_name in imgs_filename]
sample_masks = [np.expand_dims(io.imread(mask_name), -1) for mask_name in masks_filename]
image_datagen.fit(sample_imgs, augment=True, seed=seed)
mask_datagen.fit(sample_masks, augment=True, seed=seed)

image_generator = image_datagen.flow_from_directory(
    'data_fcn/train/images',
    target_size=(70, 116),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    color_mode='grayscale')

mask_generator = mask_datagen.flow_from_directory(
    'data_fcn/train/masks',
    target_size=(70, 116),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    color_mode='grayscale')

train_generator = zip(image_generator, mask_generator)

In [ ]:
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=180.,
                     horizontal_flip=True,
                     fill_mode='nearest')
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1
imgs_filename = ["data_fcn/validation/images/images/"+str(i)+".jpg" for i in range(1, 10)]
masks_filename = ["data_fcn/validation/masks/masks/"+str(i)+"_mask.jpg" for i in range(1, 10)]
sample_imgs = [np.expand_dims(io.imread(img_name), -1) for img_name in imgs_filename]
sample_masks = [np.expand_dims(io.imread(mask_name), -1) for mask_name in masks_filename]
image_datagen.fit(sample_imgs, augment=True, seed=seed)
mask_datagen.fit(sample_masks, augment=True, seed=seed)

image_generator = image_datagen.flow_from_directory(
    'data_fcn/validation/images',
    target_size=(70, 116),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    color_mode='grayscale')

mask_generator = mask_datagen.flow_from_directory(
    'data_fcn/validation/masks',
    target_size=(70, 116),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    color_mode='grayscale')

validation_generator = zip(image_generator, mask_generator)

### FCN model training on small images

In [ ]:
fcn.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=25)
fcn.save_weights('fcn.h5')

### FCN model prediction on small images demo

In [ ]:
%matplotlib inline

import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt

img = io.imread('data_fcn/train/images/images/9.jpg') # numpy.ndarray [70, 116]
mask = io.imread('data_fcn/train/masks/masks/9_mask.jpg')
img = np.expand_dims(img, 0)
img = np.expand_dims(img, -1)
pred = fcn.predict(img) # numpy.ndarray [1, 70, 116, 2]
img = np.squeeze(img)
pred = np.argmax(pred, 3) # [1, 70, 116]
pred = np.squeeze(pred, 0)

plt.figure(figsize = (15, 7))
plt.subplot(1,3,1)
plt.imshow(img)
plt.subplot(1,3,2)
plt.imshow(pred)
plt.subplot(1,3,3)
plt.imshow(mask)

### Mask detection in original images